# Tutorial pyOSRD #6
Creating custom agent

In [1]:
from pyosrd.agents.scheduler_agent import SchedulerAgent
import pandas as pd
from dataclasses import dataclass

## Creating a dumb agent
Create an agent that add 60 seconds of delay at the second zone the first train go throught.

In [2]:

class DumbSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 60s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 60.
        return extra_delays


## Test our agent on all scenarii
We can launch our custom agent on all scenarii implemented in the module OSRD.scenarii.

In [3]:
from pyosrd.agents.scheduler_agent import regulate_scenarii_with_agents
from functools import partial
from pyosrd.schedules.weights import all_steps

agent = DumbSchedulerAgent("dumb")

indicator_function = (
    lambda schedule, ref, sim:
    schedule.compute_weighted_delays(ref, all_steps(sim))
)

regulate_scenarii_with_agents("all", agent, indicator_function)

[09:46:30,609] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[09:46:30,652] [INFO]              [FullInfra] parsing infra
[09:46:30,730] [INFO]              [FullInfra] adaptation to kotlin
[09:46:30,757] [INFO]              [FullInfra] loading signals
[09:46:30,763] [INFO]              [FullInfra] building blocks
[09:46:30,768] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[09:46:30,788] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.0
[09:46:30,853] [DEBUG]             [Allowance] computing range n°1
[09:46:30,856] [DEBUG]             [Allowance]   computing section n°1
[09:46:30,856] [DEBUG]             [Allowance]   target time = 310.84094999999996
[09:46:30,857] [DEBUG]             [Allowance]     starting attempt 1
[09:46:30,858] [DEBUG]             [Allowance]     envelope time 518.958
[09:46:30,858] [DEBUG]             [Allowance]     starting attempt 2
[09:46:30,859] [DEBUG]             [Allow

,dumb
c1_delay,360.000000
c1y2_2trains_conflict,120.000000
c1y2_2trains_no_conflict,120.000000
c2y11s_conflict_20_trains,655.134424
c2y1y2y_2_trains_no_conflict,29.920409
c2y1y2y_2_trains_reorder,448.806130
c2y1y2y_2_trains_same_order,224.403065


## Compare two agents

The function regulate_scenarii_with_agents can be launched on multiple agents at the same time. It returns a pandas DataFrame containing the ponderated delays
for all scenarii.

An agent can be compared to another one using those scenarii.

In [4]:

class DumberSchedulerAgent(SchedulerAgent):
    @property
    def steps_extra_delays(self) -> pd.DataFrame:
        """As an example, make train1 stop 120s more in second zone"""
        extra_delays = self.ref_schedule.durations * 0.
        extra_delays.iloc[0, 1] = 120.
        return extra_delays

To speed up the notebook execution the next call is done only on one scenario "c1_delay".

In [6]:

agent_dumber = DumberSchedulerAgent("dumber")
regulate_scenarii_with_agents("c1_delay", [agent, agent_dumber], indicator_function)

[09:47:00,705] [INFO]  [StandaloneSimulationCommand] Loading infra: tmp/infra.json
[09:47:00,737] [INFO]              [FullInfra] parsing infra
[09:47:00,797] [INFO]              [FullInfra] adaptation to kotlin
[09:47:00,821] [INFO]              [FullInfra] loading signals
[09:47:00,826] [INFO]              [FullInfra] building blocks
[09:47:00,830] [INFO]  [StandaloneSimulationCommand] Loading input: tmp/simulation.json
[09:47:00,841] [INFO]  [StandaloneSimulationCommand] Running simulation for schedule group: group.14
[09:47:00,888] [DEBUG]             [Allowance] computing range n°1
[09:47:00,890] [DEBUG]             [Allowance]   computing section n°1
[09:47:00,890] [DEBUG]             [Allowance]   target time = 310.84094999999996
[09:47:00,890] [DEBUG]             [Allowance]     starting attempt 1
[09:47:00,891] [DEBUG]             [Allowance]     envelope time 518.958
[09:47:00,891] [DEBUG]             [Allowance]     starting attempt 2
[09:47:00,892] [DEBUG]             [Allo

,dumb,dumber
c1_delay,360.0,480.0
